In [92]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.neighbors import KNeighborsClassifier
import missingno as msno

pd.set_option('display.max_rows', None)  # 모든 행을 출력
pd.set_option('display.max_columns', None)  # 모든 열을 출력


In [93]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [121]:
train.info()
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6999 entries, 0 to 6998
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Warehouse_block      6999 non-null   object 
 1   Mode_of_Shipment     6999 non-null   object 
 2   Customer_care_calls  5423 non-null   float64
 3   Customer_rating      6999 non-null   int64  
 4   Cost_of_the_Product  6999 non-null   int64  
 5   Prior_purchases      6049 non-null   float64
 6   Product_importance   6999 non-null   object 
 7   Discount_offered     3468 non-null   float64
 8   Weight_in_gms        6999 non-null   object 
 9   Reached.on.Time_Y.N  6999 non-null   int64  
dtypes: float64(3), int64(3), object(4)
memory usage: 546.9+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Warehouse_block   

# ? 를 제외한 이상치만 수정시작

## Warehouse_block

In [122]:
# Mode_of_Shipment - shipzk 등등 ship 으로 변경
# Cost_of_the_Product - 9999이상치 제거 후 중앙값으로 대체
# Customer_rating - 평점 99 2개 평균 평점으로 대체
# Product_importance - 데이터 카테고리 이름 오타 수정

In [123]:
train.rename(columns={'Warehouse_block ': 'Warehouse_block'}, inplace=True)
test.rename(columns={'Warehouse_block ': 'Warehouse_block'}, inplace=True)


In [124]:
train['Warehouse_block']

0       C
1       F
2       F
3       F
4       D
5       B
6       A
7       F
8       F
9       C
10      B
11      F
12      F
13      C
14      F
15      B
16      B
17      B
18      A
19      C
20      C
21      C
22      F
23      C
24      A
25      A
26      F
27      F
28      F
29      F
30      D
31      F
32      F
33      D
34      B
35      D
36      F
37      D
38      A
39      A
40      B
41      C
42      F
43      C
44      F
45      D
46      B
47      D
48      A
49      F
50      F
51      C
52      C
53      D
54      F
55      A
56      F
57      B
58      B
59      A
60      D
61      A
62      B
63      B
64      D
65      D
66      D
67      A
68      C
69      A
70      F
71      C
72      D
73      F
74      F
75      F
76      C
77      F
78      D
79      F
80      F
81      A
82      A
83      F
84      F
85      B
86      D
87      F
88      F
89      F
90      C
91      C
92      C
93      C
94      D
95      B
96      F
97      B
98      D
99      F


## Mode_of_Shipment

In [125]:
# 데이터들 첫 글자 띄어쓰기 수정 Flightzk -> Flight 로 통일
train['Mode_of_Shipment'] = train['Mode_of_Shipment'].replace(' Shipzk', 'Ship')
test['Mode_of_Shipment'] = train['Mode_of_Shipment'].replace(' Shipzk', 'Ship')
train['Mode_of_Shipment'] = train['Mode_of_Shipment'].replace(' Flightzk', 'Flight')
test['Mode_of_Shipment'] = train['Mode_of_Shipment'].replace(' Flightzk', 'Flight')
train['Mode_of_Shipment'] = train['Mode_of_Shipment'].replace(' Roadzk', 'Road')
test['Mode_of_Shipment'] = train['Mode_of_Shipment'].replace(' Roadzk', 'Road')
train['Mode_of_Shipment'] = train['Mode_of_Shipment'].replace(' Road', 'Road')
train['Mode_of_Shipment'] = train['Mode_of_Shipment'].replace(' Ship', 'Ship')
train['Mode_of_Shipment'] = train['Mode_of_Shipment'].replace(' Flight', 'Flight')
test['Mode_of_Shipment'] = train['Mode_of_Shipment'].replace(' Road', 'Road')
test['Mode_of_Shipment'] = train['Mode_of_Shipment'].replace(' Ship', 'Ship')
test['Mode_of_Shipment'] = train['Mode_of_Shipment'].replace(' Flight', 'Flight')

In [126]:
train['Mode_of_Shipment'].unique()

array(['Ship', 'Flight', 'Road', '?'], dtype=object)

In [127]:
test['Mode_of_Shipment'].unique()

array(['Ship', 'Flight', 'Road', '?'], dtype=object)

## Customer_rating

In [128]:
# max 에서 99 이상치 총 2개 
median_cost = train['Customer_rating'].median()
train['Customer_rating'].replace(99, median_cost, inplace=True)
test['Customer_rating'].replace(99, median_cost, inplace=True)

In [129]:
train['Customer_rating'].describe()

count    6999.000000
mean        2.996285
std         1.410161
min         1.000000
25%         2.000000
50%         3.000000
75%         4.000000
max         5.000000
Name: Customer_rating, dtype: float64

In [130]:
test['Customer_rating'].describe()

count    4000.00000
mean        2.98125
std         1.41912
min         1.00000
25%         2.00000
50%         3.00000
75%         4.00000
max         5.00000
Name: Customer_rating, dtype: float64

## Cost_of_the_Product  

In [131]:
# Cost_of_the_Product 9999 이상치 3개 중앙값 대체 
median_cost = train['Cost_of_the_Product'].median()
train['Cost_of_the_Product'].replace(9999, median_cost, inplace=True)
test['Cost_of_the_Product'].replace(9999, median_cost, inplace=True)

In [132]:
train['Cost_of_the_Product'].describe()

count    6999.000000
mean      210.055008
std        48.220091
min        96.000000
25%       169.000000
50%       214.000000
75%       251.000000
max       310.000000
Name: Cost_of_the_Product, dtype: float64

In [133]:
test['Cost_of_the_Product'].describe()

count    4000.000000
mean      210.420250
std        47.755542
min        96.000000
25%       170.000000
50%       214.000000
75%       251.000000
max       310.000000
Name: Cost_of_the_Product, dtype: float64

## Product_importance

- 컬럼 네임 수정

In [134]:
train['Product_importance'].replace('mediumm', 'medium', inplace=True)
train['Product_importance'].replace('loww', 'low', inplace=True)
train['Product_importance'].replace('highh', 'high', inplace=True)
test['Product_importance'].replace('mediumm', 'medium', inplace=True)
test['Product_importance'].replace('loww', 'low', inplace=True)
test['Product_importance'].replace('highh', 'high', inplace=True)

In [135]:
train['Product_importance'].unique()

array(['high', 'medium', 'low', '?'], dtype=object)

In [136]:
test['Product_importance'].unique()

array(['high', 'medium', 'low', '?'], dtype=object)

In [137]:
# 카테고리 순서 변경
train['Product_importance'] = train['Product_importance'].map({'low': 'high', 'medium': 'medium', 'high': 'low', '?': '?'})


In [138]:
train['Product_importance'].info()
train['Product_importance'].describe()

<class 'pandas.core.series.Series'>
RangeIndex: 6999 entries, 0 to 6998
Series name: Product_importance
Non-Null Count  Dtype 
--------------  ----- 
6999 non-null   object
dtypes: object(1)
memory usage: 54.8+ KB


count     6999
unique       4
top        low
freq      3345
Name: Product_importance, dtype: object

In [139]:
test['Product_importance'].info()
test['Product_importance'].describe()

<class 'pandas.core.series.Series'>
RangeIndex: 4000 entries, 0 to 3999
Series name: Product_importance
Non-Null Count  Dtype 
--------------  ----- 
4000 non-null   object
dtypes: object(1)
memory usage: 31.4+ KB


count     4000
unique       4
top        low
freq      1873
Name: Product_importance, dtype: object

# ? 를 제외한 이상치 처리 완료

# 추가 필요없는 컬럼 제거

In [140]:
train.drop('ID', axis=1, inplace=True)
train.drop('Gender', axis=1, inplace=True)
test.drop('ID', axis=1, inplace=True)
test.drop('Gender', axis=1, inplace=True)

KeyError: "['ID'] not found in axis"

In [141]:
set(train.columns) - set(test.columns)

{'Reached.on.Time_Y.N'}

In [142]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6999 entries, 0 to 6998
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Warehouse_block      6999 non-null   object 
 1   Mode_of_Shipment     6999 non-null   object 
 2   Customer_care_calls  5423 non-null   float64
 3   Customer_rating      6999 non-null   int64  
 4   Cost_of_the_Product  6999 non-null   int64  
 5   Prior_purchases      6049 non-null   float64
 6   Product_importance   6999 non-null   object 
 7   Discount_offered     3468 non-null   float64
 8   Weight_in_gms        6999 non-null   object 
 9   Reached.on.Time_Y.N  6999 non-null   int64  
dtypes: float64(3), int64(3), object(4)
memory usage: 546.9+ KB


In [143]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Warehouse_block      4000 non-null   object 
 1   Mode_of_Shipment     4000 non-null   object 
 2   Customer_care_calls  3144 non-null   float64
 3   Customer_rating      4000 non-null   int64  
 4   Cost_of_the_Product  4000 non-null   int64  
 5   Prior_purchases      3633 non-null   float64
 6   Product_importance   4000 non-null   object 
 7   Discount_offered     2085 non-null   float64
 8   Weight_in_gms        4000 non-null   object 
dtypes: float64(3), int64(2), object(4)
memory usage: 281.4+ KB


In [144]:
train.to_csv('train_fix_1.csv', index=False)
test.to_csv('test_fix_1.csv', index=False)

In [120]:
train.info()
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6999 entries, 0 to 6998
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Warehouse_block      6999 non-null   object 
 1   Mode_of_Shipment     6999 non-null   object 
 2   Customer_care_calls  5423 non-null   float64
 3   Customer_rating      6999 non-null   int64  
 4   Cost_of_the_Product  6999 non-null   int64  
 5   Prior_purchases      6049 non-null   float64
 6   Product_importance   6999 non-null   object 
 7   Discount_offered     3468 non-null   float64
 8   Weight_in_gms        6999 non-null   object 
 9   Reached.on.Time_Y.N  6999 non-null   int64  
dtypes: float64(3), int64(3), object(4)
memory usage: 546.9+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Warehouse_block   